<a href="https://colab.research.google.com/github/ajits-github/Advanced_Python/blob/main/Advanced_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multithreading

Life-cycle of a Thread:
  * **Create** by calling the constructor Thread and passing the function to the 'target' keyword along with the arguments (in the 'args' keyword argument of Thread, if the function expects so).
  * **Start** the thread i.e. transition to 'run' state by calling 'start()' on thread object.
  * **Terminated** thread, which happens if the subsequent new thread created by main thread has completed the executio of the program or raised exception.

In [ ]:
from threading import Thread
from time import sleep

def task():
  sleep(1)
  print("Inside task")

def new_task():
  sleep(1)
  print("Inside NEW task")

thread1 = Thread(target=task)
thread1.start()
thread1.join(timeout=2)

thread2 = Thread(target=new_task)
thread2.start()
# thread2.join(timeout=2)
# thread2.join(timeout=0.1)

if thread2.is_alive():
  print("Thread 2 is still running")
else:
  print("Thread 2 is not running")

Inside task
Thread 2 is still running


In the above code, few things that need discussion:
  * Can we call new_task() funtion on thread1 at the same time as we called task()?
    * Nope, it's not possible to call multiple target functions on a single thread. We need to declare separate threads for sifferent functions.

  * What is the purpose of join() on a Thread?
    * join() waits for a thread to complete its execution. This duration of waiting can be restricted by using 'timeout' in join() which allows the later code to run if, for example, the timeout is smaller than sleep time inside the function called on that thread. This is the same effect achived by not using join(). The results will be same in both the cases as it can be seen from the above output where Thread 2 is still running though it didn't print the statement inside new_task() function. Observe the outputs by commenting line 18 and 19, and then uncomment line 19. However, now comment line 19 and uncomment 18, you will see the difference as why we need join().





---



We use Multithreading for I/O tasks which requires waiting for external events and not for the CPU bound tasks. For that, we use Multiprocessing module from Python. Thread is the smallest unti of program. A process will have at least on thread and so many threads, under one process, share the memory space.

##Multiprocessing

##Decorators

In [ ]:
# def outer_func(arg_outer_def):
def outer_func():
  def outer_func_2(func):
    def real_func(*args, **kwargs):
      # def inner_func(arg1):
      def inner_func(*args, **kwargs):
        # print("Before calling..", type(arg1))
        print("Before calling..", args, kwargs)
        # arg1 = str(arg1) + arg_outer_def
        # func(arg1)
        func(args)
        # print()
        # return func(arg1)
      return inner_func
    return real_func
  return outer_func_2

# @outer_func(arg_outer_def="static")
@outer_func()
def after_func(arg1="test"):
  # if isinstance(arg1, str):
  print("hELLLOOO")
  if isinstance(arg1, list):
    print("As expected, arg is converted to string: ", type(arg1))
    print(arg1)


# after_func([4,5,6])
after_func([4,5,6], arg1="hello")

<function __main__.outer_func.<locals>.outer_func_2.<locals>.real_func.<locals>.inner_func(*args, **kwargs)>

In [ ]:
def outer_func():
    def inner1(func):
        def inner2(*args, **kwargs):
            def inner3(args):
                def inner4(args):
                    def inner5(args):
                        print("Inside inner5")
                        func(*args, **kwargs)
                        print("Exiting inner5")
                    return inner5
                return inner4
            return inner3
        return inner2
    return inner1

@outer_func()
def decorated_function(arg1="test"):
# def decorated_function(arg1):
    print("Decorated function with arg1:", arg1)

decorated_function("Hello!")


<function __main__.outer_func.<locals>.inner1.<locals>.inner2.<locals>.inner3(args)>

In [ ]:
def outer_func(func):
    def inner1(*args, **kwargs):
        def inner2(*args, **kwargs):
            print("Inside inner2")
            func(*args, **kwargs)
            print("Exiting inner2")
        return inner2
    return inner1

@outer_func
def decorated_function(arg1="test"):
    print("Decorated function with arg1:", arg1)

# Call the decorated function
decorated_function("Hello!")


<function __main__.outer_func.<locals>.inner1.<locals>.inner2(*args, **kwargs)>

In [ ]:
def outer_func(who):
    def inner_func():
        print(f"Hello, {who}")
    inner_func()


outer_func("World!")

Hello, World!


Important points about decorators:
* The signature of the most inner function and the function which is being decorated must be same or with variable args.

What do we mean by something is callable?
- Check the below code which is having outer and inner functions. When the outer function is called it just assigns the object of inner function so that later on the same can be invoked with proper signature of inner function. For example, if we comment line 4 and 5, and uncomment the lines 7,8 and 9, the obj_outer still holds the object of inner_func but if we change the call to a constant instead of a function inside out_function body (as currently it is showing), then you get the error which should be as the string object which is being passed upon call is no more a function but just a constant.

In [ ]:
def out_function(out_args):
  print("hello, outside: ", out_args)

  value = out_args
  return value

  # def inner_func(*args):
  #   return "hello, inside " + str(args) + str(out_args)
  # return inner_func



obj_outer = out_function("_outercall_")
obj_outer("_innercall_")


hello, outside:  _outercall_


TypeError: ignored

The use cases for Python decorators are varied. Here are some of them:

* Debugging
* Caching
* Logging
* Timing

Functools.wraps decorator:

In [ ]:
import functools

def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Decorator is called")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def my_function():
    """This is the docstring of my_function."""
    print("Inside my_function")

print("Name of the decorated function:", my_function.__name__)
print("Docstring of the decorated function:", my_function.__doc__)


Name of the decorated function: wrapper
Docstring of the decorated function: None


In the example above, when we run the code, we'll notice that the name and docstring of the my_function are overwritten by the wrapper function inside the my_decorator.

To preserve the original metadata of the decorated function, we can use functools.wraps. Here's how we can modify the decorator to use functools.wraps:

In [ ]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Decorator is called")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def my_function():
    """This is the docstring of my_function."""
    print("Inside my_function")

print("Name of the decorated function:", my_function.__name__)
print("Docstring of the decorated function:", my_function.__doc__)


Name of the decorated function: my_function
Docstring of the decorated function: This is the docstring of my_function.


By using functools.wraps, we ensure that the decorated function retains its original metadata, making our code more maintainable and easier to understand during debugging and analysis.